In [30]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
import networkx as nx
import matplotlib.pyplot as plt
import copy
import elitism
import json

In [31]:
with open('./data.json', 'r') as file:
    data = json.load(file)
data = data[0]
data

{'stations': {'Златоуст (1)': ['0', '38', '38', '25'],
  'Кыштым (2)': ['26', '0', '7', '34'],
  'Миасс (3)': ['15', '5', '0', '27'],
  'Муслюмово (4)': ['12', '39', '1', '0']},
 'full_timetable': {'1': {'route': ['1', '3', '2'],
   'free_carriage': ['21', '21'],
   'timetable': ['02:00 - 02:38', '04:48 - 06:11', '13:35 - 13:58']},
  '2': {'route': ['2', '3'],
   'free_carriage': ['10'],
   'timetable': ['02:00 - 02:40', '04:44 - 06:09']}}}

In [32]:
stations = [list(map(int, x)) for x in data['stations'].values()]
stations

[[0, 38, 38, 25], [26, 0, 7, 34], [15, 5, 0, 27], [12, 39, 1, 0]]

In [33]:
trains = [x for x in data['full_timetable'].values()]
trains

[{'route': ['1', '3', '2'],
  'free_carriage': ['21', '21'],
  'timetable': ['02:00 - 02:38', '04:48 - 06:11', '13:35 - 13:58']},
 {'route': ['2', '3'],
  'free_carriage': ['10'],
  'timetable': ['02:00 - 02:40', '04:44 - 06:09']}]

In [34]:
for train in trains:
    train['free_carriage'].append(0)

In [35]:
timetable = ['02:44 - 03:14',
   '05:24 - 05:56',
   '09:34 - 11:00',
   '12:22 - 13:05',
   '23:43 - 00:34']

In [36]:
def convert_train_to_time_list(train_list):
    train_by_time = []
    for train_id in range(len(train_list)):
        time_list = train_list[train_id]['timetable']

        for i, time in enumerate(time_list):
            before_after = time.split('-')  # [05:24, 05:56]

            for j, rout_part in enumerate(before_after):
                free_carriage = int(train_list[train_id]['free_carriage'][i]) if j == 1 else None
                free_carriage = None if free_carriage == 0 else free_carriage
                hour, minut = [int(x) for x in rout_part.split(':')]
                new_t = hour * 100 + minut
                if i > 0 and train_by_time[-1]['time'] > new_t:
                    new_t *= 1000
                
                train_by_time.append({
                    'id': train_id, 
                    'time': new_t,
                    'st': int(train_list[train_id]['route'][i]), 
                    'free_carriage': free_carriage,
                    'onboard': [0] * 4
                })
    return train_by_time

In [37]:
train_to_time = convert_train_to_time_list(trains)
train_to_time = sorted(train_to_time, key = lambda x: x['time'])
train_to_time

[{'id': 0,
  'time': 200,
  'st': 1,
  'free_carriage': None,
  'onboard': [0, 0, 0, 0]},
 {'id': 1,
  'time': 200,
  'st': 2,
  'free_carriage': None,
  'onboard': [0, 0, 0, 0]},
 {'id': 0, 'time': 238, 'st': 1, 'free_carriage': 21, 'onboard': [0, 0, 0, 0]},
 {'id': 1, 'time': 240, 'st': 2, 'free_carriage': 10, 'onboard': [0, 0, 0, 0]},
 {'id': 1,
  'time': 444,
  'st': 3,
  'free_carriage': None,
  'onboard': [0, 0, 0, 0]},
 {'id': 0,
  'time': 448,
  'st': 3,
  'free_carriage': None,
  'onboard': [0, 0, 0, 0]},
 {'id': 1,
  'time': 609,
  'st': 3,
  'free_carriage': None,
  'onboard': [0, 0, 0, 0]},
 {'id': 0, 'time': 611, 'st': 3, 'free_carriage': 21, 'onboard': [0, 0, 0, 0]},
 {'id': 0,
  'time': 1335,
  'st': 2,
  'free_carriage': None,
  'onboard': [0, 0, 0, 0]},
 {'id': 0,
  'time': 1358,
  'st': 2,
  'free_carriage': None,
  'onboard': [0, 0, 0, 0]}]

In [38]:
HROM_LENGHT = sum([1 for x in train_to_time if x['free_carriage'] is not None])
VAGON_COUNT = sum([sum(i) for i in stations])
TRAIN_COUNT = 5


# константы генетического алгоритма
POPULATION_SIZE = 100   # количество индивидуумов в популяции
P_CROSSOVER = 0.9       # вероятность скрещивания
P_MUTATION = 0.2        # вероятность мутации индивидуума
MAX_GENERATIONS = 200    # максимальное количество поколений
HALL_OF_FAME_SIZE = 10
INDIVID_SIZE = 4

MAX_INDIVID = max([int(y) for x in [i['free_carriage'] for i in trains] for y in x])

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [39]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, Dropout, Flatten
import keras

In [40]:
TRAIN_LEAVE = 2
TRAIN_RECIEVE = 2
FREE_CARRIAGE = len(stations)

first_count = len(stations) + (TRAIN_LEAVE + FREE_CARRIAGE) + (TRAIN_RECIEVE  + len(stations) * TRAIN_RECIEVE)

model = keras.Sequential(
    [
        Dense(first_count, activation="relu", input_shape=(first_count,)),
        # Dropout(0.2),
        Dense(first_count // 2, activation="relu"),
        # Dropout(0.2),
        Dense(len(stations), activation='softmax'),
    ]
)

In [41]:

import tensorflow as tf
optimiser = tf.keras.optimizers.RMSprop()
model.compile(optimizer=optimiser, loss='mse', metrics=['accuracy'])

model.build()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 20)                420       
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 4)                 44        
                                                                 
Total params: 674 (2.63 KB)
Trainable params: 674 (2.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
for layer in model.layers:
    print(layer.get_weights()[0].shape)

(20, 20)
(20, 10)
(10, 4)


In [43]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

/home/atikin/anaconda3/envs/dataAnalytics/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/atikin/anaconda3/envs/dataAnalytics/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [44]:
def create_individ(count):
    res = np.random.rand(count) * 2 - 1
    return creator.Individual(res)

In [45]:
toolbox = base.Toolbox()
toolbox.register("individualCreator", create_individ, model.count_params())
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

In [46]:
population = toolbox.populationCreator(n=POPULATION_SIZE)
population[0]

[0.6256513617399264,
 -0.20005380685986252,
 0.2059476586629927,
 -0.42969895423862514,
 -0.0929113034619431,
 0.7958975492169518,
 -0.3947327590112193,
 -0.1225042956367357,
 -0.989990108376477,
 0.8815018304091153,
 -0.5246274673903513,
 0.6278988550469999,
 -0.034521835683592794,
 0.2960824260224839,
 -0.8626120632948158,
 0.42487259111063436,
 -0.2535115711250244,
 0.8907971401792278,
 0.6925617771802208,
 -0.024830049809513755,
 -0.46921752654118576,
 0.6524580438419658,
 -0.2181319218085669,
 -0.4691282066033191,
 -0.5385678519037083,
 0.394114453067661,
 -0.7690953548027448,
 -0.7697302310181393,
 -0.16669353061862946,
 -0.07844328232187348,
 0.8553802381486371,
 -0.1301294083852944,
 0.38422871292934757,
 0.2187805399919467,
 -0.41638448448366905,
 -0.44708098569312127,
 0.9955570536957128,
 0.21086431599887967,
 0.09929698528631792,
 -0.7656447000574189,
 0.7983046646556908,
 -0.39049128712607484,
 -0.5821202705227588,
 -0.6759852957586079,
 -0.12407932960560086,
 0.3405403005

In [47]:
def get_top_train(time, st, count, leave = False):
    res = []
    for train in train_to_time:
        if train['time'] > time and train['st'] == st:
            if leave is False and train['free_carriage'] is None:
                res.append(train['id'])
                res.extend(train['onboard'])
            if leave is True and train['free_carriage'] is not None:
                res.append(train['id'])
                res.append(train['free_carriage'])

            if len(res) == count:
                return res
    i = len(res)
    while i <= count:
        res.append(0)
    
    return res

In [48]:
PENALTY_COST = 200

def shedule_cost(individual, model):
    individual = np.array(individual)
    current_cost = 0
    penalty_count = 0
    new_stations = copy.deepcopy(stations)

    weights_shapes = [layer.get_weights()[0].shape for layer in model.layers]
    print(weights_shapes)
    split_weights = np.split(individual, [np.prod(shape) for shape in weights_shapes])
    # weights = []
    # weights = np.array_split(individual, [20, 40, 50, 60, 64])
    print(split_weights)
    # weights = np.random.rand(20, 20)


    # model.set_weights(split_weights)
    for layer, weights in zip(model.layers, split_weights):
        # print([weights.reshape(shape) for shape in layer.get_weights()[0].shape])
        layer.set_weights([weights.reshape(shape) for shape in layer.get_weights()[0].shape])

    for time in range(len(train_to_time)):
        train = train_to_time[time]
        if train['free_carriage'] is None:
            continue
        
        enter = new_stations[train['st']]
        enter += get_top_train(time, train['st'], TRAIN_LEAVE)
        enter += get_top_train(time, train['st'], TRAIN_LEAVE, True)

        pred = model.predict(enter)

        print(pred)



        

    # model.set_weights(individual)

    current_cost += penalty_count * PENALTY_COST
    return current_cost,

In [49]:
# model.layers[5].get_weights().shape

In [50]:
shedule_cost(population[0], model)

[(20, 20), (20, 10), (10, 4)]
[array([ 0.62565136, -0.20005381,  0.20594766, -0.42969895, -0.0929113 ,
        0.79589755, -0.39473276, -0.1225043 , -0.98999011,  0.88150183,
       -0.52462747,  0.62789886, -0.03452184,  0.29608243, -0.86261206,
        0.42487259, -0.25351157,  0.89079714,  0.69256178, -0.02483005,
       -0.46921753,  0.65245804, -0.21813192, -0.46912821, -0.53856785,
        0.39411445, -0.76909535, -0.76973023, -0.16669353, -0.07844328,
        0.85538024, -0.13012941,  0.38422871,  0.21878054, -0.41638448,
       -0.44708099,  0.99555705,  0.21086432,  0.09929699, -0.7656447 ,
        0.79830466, -0.39049129, -0.58212027, -0.6759853 , -0.12407933,
        0.3405403 ,  0.38442151, -0.66552346,  0.10513262,  0.85072582,
       -0.00924492,  0.17016334,  0.76881579,  0.41159333,  0.79542994,
        0.90576263,  0.03814112,  0.45833937, -0.53856858, -0.85705141,
        0.52646003, -0.63399722, -0.98272165, -0.17692807, -0.6115617 ,
       -0.95961556, -0.46812217, 

ValueError: cannot reshape array of size 400 into shape (20,)